In [1]:
# NLP_mvp_refactored.py

# ## NLP: Análisis de Sentimiento Robusto

# #### 1. Importación de Librerías
# Se importan todas las librerías necesarias al inicio para una mejor gestión de dependencias.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
#from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import os


In [2]:

# Descargar e instalar el modelo de spaCy si no está presente
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print('Descargando el modelo en_core_web_sm de spaCy...')
    # La siguiente línea es para ejecutar en un entorno de notebook/terminal
    os.system('python -m spacy download en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')



In [3]:
import random

# #### 2. Carga y Exploración de Datos (con muestra aleatoria)
# Leemos una muestra aleatoria para acelerar el prototipado sin sesgos.

try:
    # --- PARÁMETROS PARA EL MUESTREO ---
    total_filas = 1600000
    filas_a_leer = 1000
    probabilidad = filas_a_leer / total_filas

    # --- LÍNEA MODIFICADA ---
    # Usamos skiprows con una función lambda para muestreo aleatorio
    df = pd.read_csv('data/training.1600000.processed.noemoticon.csv',
        encoding='latin-1',
        header=None,
        # Omitir una fila si un número aleatorio es mayor que nuestra probabilidad
        skiprows=lambda i: i > 0 and random.random() > probabilidad
    )
    print(f'Dataset cargado exitosamente (muestra aleatoria de ~{filas_a_leer} filas).')
    print(df.head())

    # Asignar nombres a las columnas
    df.columns = ['Sentiment', 'id', 'date', 'query', 'user', 'Text']
    
    # Eliminar columnas innecesarias
    cols_to_drop = ['id', 'date', 'query']
    df = df.drop(columns=cols_to_drop)
    
    print('\nPrimeras 10 filas de la muestra aleatoria:')
    print(df.head(10))
    
    print(f'\nDimensiones de la muestra: {df.shape}')

except FileNotFoundError:
    print('Error: El archivo no fue encontrado.')
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Dataset cargado exitosamente (muestra aleatoria de ~1000 filas).
   0           1                             2         3                4  \
0  0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1  0  1468511278  Tue Apr 07 02:07:04 PDT 2009  NO_QUERY      Mike_Rhodes   
2  0  1468842850  Tue Apr 07 04:01:12 PDT 2009  NO_QUERY      nameskieren   
3  0  1469007159  Tue Apr 07 04:48:44 PDT 2009  NO_QUERY        mattmagic   
4  0  1469125450  Tue Apr 07 05:18:10 PDT 2009  NO_QUERY     LoneWolf2003   

                                                   5  
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1             i have to wake up in 5 hours  laameeee  
2                             has a really bad cold   
3  ARGHHH spent all yesterday doing a DPS for WMB...  
4   thats y i never play uno  all the green cards go  

Primeras 10 filas de la muestra aleatoria:
   Sentiment             user  \
0          0  _TheSpecialOne_   
1          0      Mike_Rhodes   

In [4]:
# #### 3. Mapeo de Sentimientos (Simplificado)
# Mapeamos los valores numéricos del sentimiento (0 y 4) a etiquetas de texto.
if 'df' in locals():
    # El valor 0 es Negativo y 4 es Positivo.
    df['Sentiment'] = df['Sentiment'].map({0: 'Negative', 4: 'Positive'})
    
    # Creamos la copia después del mapeo
    df_copy = df.copy()

    print('Distribución de sentimientos tras el mapeo:')
    print(df['Sentiment'].value_counts())

    

Distribución de sentimientos tras el mapeo:
Sentiment
Negative    533
Positive    486
Name: count, dtype: int64


In [5]:

# #### 4. Preprocesamiento de Texto
# Función de limpieza robusta que maneja posibles valores no textuales, convierte a minúsculas, elimina caracteres no deseados, lematiza y quita *stop words*.
def limpieza(texto):
    """Realiza una limpieza completa del texto, incluyendo manejo de casos nulos."""
    if not isinstance(texto, str):
        return ""
    
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúüñ\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    # Procesar con spaCy solo si el texto no está vacío
    if not texto:
        return ""
        
    doc = nlp(texto)
    texto_limpio = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    
    return texto_limpio

# ### Preparación de Datos para Modelado
if 'df_copy' in locals():
    X = df_copy['Text'].apply(limpieza)
    
    # Mapeamos 'Negative' a -1 y 'Positive' a 1
    y = df_copy['Sentiment'].map({'Negative': -1, 'Positive': 1})

    # Eliminar filas donde 'y' podría ser NaN si hubo valores de sentimiento inesperados
    # y asegurarse de que los índices coincidan
    valid_indices = y.notna()
    X = X[valid_indices]
    y = y[valid_indices]

    # Dividir los datos, estratificando por 'y' para mantener la proporción de clases
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    print(f"\n- Data para modelado preparada -")
    print(f"Tamaño de X_train: {X_train.shape[0]}")
    print(f"Tamaño de X_test: {X_test.shape[0]}")
    print(f"\nDistribución en y_train:\n{y_train.value_counts()}")


- Data para modelado preparada -
Tamaño de X_train: 713
Tamaño de X_test: 306

Distribución en y_train:
Sentiment
-1    373
 1    340
Name: count, dtype: int64


In [6]:

# ### Modelo 3: TF-IDF + Regresión Logística
# Este método pondera la importancia de las palabras según su frecuencia en el documento y en todo el corpus (TF-IDF). Se combina con una Regresión Logística, un modelo lineal robusto para clasificación.
if 'X_train' in locals():
    # Vectorización con TF-IDF
    vectorizer_tfidf = TfidfVectorizer()
    X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
    X_test_tfidf = vectorizer_tfidf.transform(X_test)

    # Entrenamiento del modelo de Regresión Logística
    # Se aumenta max_iter para asegurar la convergencia
    model_lr = LogisticRegression(random_state=42, max_iter=1000)
    model_lr.fit(X_train_tfidf, y_train)

    # Predicciones
    y_train_pred_lr = model_lr.predict(X_train_tfidf)
    y_test_pred_lr = model_lr.predict(X_test_tfidf)

    # Reporte de Clasificación
    print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---")
    print(classification_report(y_train, y_train_pred_lr))
    print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---")
    print(classification_report(y_test, y_test_pred_lr))


--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---
              precision    recall  f1-score   support

          -1       0.94      0.98      0.96       373
           1       0.98      0.93      0.96       340

    accuracy                           0.96       713
   macro avg       0.96      0.96      0.96       713
weighted avg       0.96      0.96      0.96       713


--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---
              precision    recall  f1-score   support

          -1       0.66      0.72      0.69       160
           1       0.66      0.60      0.63       146

    accuracy                           0.66       306
   macro avg       0.66      0.66      0.66       306
weighted avg       0.66      0.66      0.66       306



In [7]:
# Vectorización TF-IDF Optimizada
vectorizer_tfidf_opt = TfidfVectorizer(
    ngram_range=(1, 2),  # Considera palabras individuales y pares de palabras (bigramas)
    max_features=20000,  # Limita el vocabulario a las 20,000 características más importantes
    min_df=5,            # La palabra debe aparecer en al menos 5 documentos
    max_df=0.7           # La palabra no debe aparecer en más del 70% de los documentos
)

# Luego usas este nuevo vectorizador para transformar tus datos
X_train_tfidf_opt = vectorizer_tfidf_opt.fit_transform(X_train)
X_test_tfidf_opt = vectorizer_tfidf_opt.transform(X_test)

# Entrenamiento del modelo de Regresión Logística
# Se aumenta max_iter para asegurar la convergencia
model_lr = LogisticRegression(random_state=42, max_iter=1000)
model_lr.fit(X_train_tfidf_opt, y_train)

# Predicciones
y_train_pred_lr = model_lr.predict(X_train_tfidf_opt)
y_test_pred_lr = model_lr.predict(X_test_tfidf_opt)

# Reporte de Clasificación
print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---")
print(classification_report(y_train, y_train_pred_lr))
print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---")
print(classification_report(y_test, y_test_pred_lr))

import joblib
import os

# --- CÓDIGO A AGREGAR ---

# Crear el directorio 'model' si no existe
output_dir = 'model'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Guardar el vectorizador TF-IDF optimizado
joblib.dump(vectorizer_tfidf_opt, os.path.join(output_dir, 'schimizzi_vectorizer.joblib'))

# Guardar el modelo de Regresión Logística entrenado
joblib.dump(model_lr, os.path.join(output_dir, 'schimizzi_modelo_s.joblib'))

print("\\nModelo y vectorizador guardados exitosamente en la carpeta 'model'.")


--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---
              precision    recall  f1-score   support

          -1       0.74      0.86      0.80       373
           1       0.82      0.67      0.73       340

    accuracy                           0.77       713
   macro avg       0.78      0.77      0.77       713
weighted avg       0.78      0.77      0.77       713


--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---
              precision    recall  f1-score   support

          -1       0.62      0.69      0.65       160
           1       0.61      0.54      0.57       146

    accuracy                           0.62       306
   macro avg       0.62      0.61      0.61       306
weighted avg       0.62      0.62      0.62       306

\nModelo y vectorizador guardados exitosamente en la carpeta 'model'.


Guardar los archivos

Cargar los archivos para usarlos

In [8]:
import joblib

# 1. Cargar los objetos guardados
vectorizer_cargado = joblib.load('model/schimizzi_vectorizer.joblib')
modelo_cargado = joblib.load('model/schimizzi_modelo_s.joblib')

# 2. Nuevos datos para predecir
nuevos_datos = [
    "I love this product, it's the best thing I've ever bought!", # Positivo
    "This is a terrible experience, I'm so disappointed.",     # Negativo
]

# 3. Usar el VECTORIZER CARGADO para transformar el texto
nuevos_datos_transformados = vectorizer_cargado.transform(nuevos_datos)

# 4. Usar el MODELO CARGADO para hacer la predicción
predicciones = modelo_cargado.predict(nuevos_datos_transformados)

# 5. Interpretar el resultado (-1: Negativo, 1: Positivo)
for texto, sentimiento in zip(nuevos_datos, predicciones):
    resultado = "Positivo" if sentimiento == 1 else "Negativo"
    print(f"Texto: '{texto}' -> Sentimiento Predicho: {resultado}")

Texto: 'I love this product, it's the best thing I've ever bought!' -> Sentimiento Predicho: Positivo
Texto: 'This is a terrible experience, I'm so disappointed.' -> Sentimiento Predicho: Negativo


In [9]:
import pandas as pd
import joblib

# 1. Cargar los objetos guardados
try:
    vectorizer_cargado = joblib.load('model/schimizzi_vectorizer.joblib')
    modelo_cargado = joblib.load('model/schimizzi_modelo_s.joblib')
    print("Vectorizer y modelo cargados correctamente.")

    # 2. Leer los datos desde el archivo CSV (corregido)
    predict_csv = 'data/testdata.manual.2009.06.14.csv'
    # Cargar sin encabezado y asignar nombres a las columnas
    df_nuevos = pd.read_csv(predict_csv, header=None, encoding='latin-1')
    df_nuevos.columns = ['Sentiment', 'id', 'date', 'query', 'user', 'Text']

    # --- ¡PUNTO CLAVE CORREGIDO! ---
    # Extraer la columna de texto correcta ('Text')
    nuevos_datos = df_nuevos['Text'].tolist()
    
    print(f"\\nFrases cargadas desde la columna 'Text' del archivo '{predict_csv}':")
    for frase in nuevos_datos[:5]: # Mostramos solo las primeras 5 para brevedad
        print(f"- {frase}")

    # (El resto del código para transformar y predecir permanece igual)
    # ...

except FileNotFoundError:
    print("\\nError: No se encontraron los archivos del modelo o el archivo CSV.")
except KeyError:
    print(f"\\nError: La columna 'Text' no se encontró en el archivo CSV.")
    print("Por favor, verifica el nombre de la columna en tu archivo.")
except Exception as e:
    print(f"\\nOcurrió un error inesperado: {e}")

Vectorizer y modelo cargados correctamente.
\nFrases cargadas desde la columna 'Text' del archivo 'data/testdata.manual.2009.06.14.csv':
- @stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.
- Reading my kindle2...  Love it... Lee childs is good read.
- Ok, first assesment of the #kindle2 ...it fucking rocks!!!
- @kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)
- @mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)


In [21]:
import pandas as pd

# --- LÍNEAS MODIFICADAS ---
# Establecer un ancho máximo razonable para las columnas
pd.set_option('display.max_colwidth', 100)
# Establecer el ancho total de la pantalla para la tabla
pd.set_option('display.width', 200)


# --- Parte 1: Imprimir algunas predicciones ---

print("--- Muestra de Predicciones del Conjunto de Prueba ---")

# (El resto del código de esta sección no necesita cambios)
# ... (código para imprimir las 5 muestras) ...
indices_muestra = X_test.head(10).index
textos_originales = df_copy.loc[indices_muestra]['Text']
muestras_transformadas = vectorizer_tfidf_opt.transform(X_test.head(10))
predicciones_muestra = model_lr.predict(muestras_transformadas)
etiquetas_reales = y_test.head(10).values
mapa_sentimiento = {-1: "Negativo", 1: "Positivo"}

for i in range(10):
    texto = textos_originales.iloc[i]
    prediccion = mapa_sentimiento[predicciones_muestra[i]]
    etiqueta_real = mapa_sentimiento[etiquetas_reales[i]]
    print(f"\nTweet: {texto}")
    print(f"  ➡️ Predicción: {prediccion} (Etiqueta Real: {etiqueta_real})")


# --- Parte 2: Guardar todos los resultados en un archivo Excel ---

print("\n\n--- Guardando todas las predicciones en Excel ---")

# (El resto del código de esta sección no necesita cambios)
# ... (código para guardar en Excel e imprimir el head) ...
y_test_predicciones = model_lr.predict(vectorizer_tfidf_opt.transform(X_test))
textos_originales_test = df_copy.loc[X_test.index]['Text']
df_resultados = pd.DataFrame({
    'Tweet': textos_originales_test,
    'Sentimiento_Predicho_Num': y_test_predicciones
})
df_resultados['Sentimiento_Predicho'] = df_resultados['Sentimiento_Predicho_Num'].map(mapa_sentimiento)
df_final_excel = df_resultados[['Tweet', 'Sentimiento_Predicho']]
nombre_archivo_excel = 'predicciones_sentimientos.xlsx'
df_final_excel.to_excel(nombre_archivo_excel, index=False, engine='openpyxl')

print(f"\n✅ ¡Éxito! Se han guardado {len(df_final_excel)} predicciones en el archivo '{nombre_archivo_excel}'.")
print("\nPrimeras filas del archivo guardado:")
print(df_final_excel.head())

--- Muestra de Predicciones del Conjunto de Prueba ---

Tweet: let's make out 
  ➡️ Predicción: Positivo (Etiqueta Real: Positivo)

Tweet: @grandmabomb Mine too. The stress is giving me a fever. Trouble is... I can't take a day off 
  ➡️ Predicción: Negativo (Etiqueta Real: Negativo)

Tweet: @ally13524 hey if you have a friend request on OMGPOP it's just little old me 
  ➡️ Predicción: Negativo (Etiqueta Real: Positivo)

Tweet: @guyfromucla u have two jobs that r unpaid...   
  ➡️ Predicción: Negativo (Etiqueta Real: Negativo)

Tweet: @pearlofthesea_ love sookis friend, the black one 
  ➡️ Predicción: Positivo (Etiqueta Real: Positivo)

Tweet: Had a good sleep!  I love my Ilan totally!  30 days love 
  ➡️ Predicción: Positivo (Etiqueta Real: Positivo)

Tweet: Chillin at the crib, about to call Kenzie 
  ➡️ Predicción: Negativo (Etiqueta Real: Positivo)

Tweet: Ugh fun concert  screen cracked on phone again because it fell... =\
  ➡️ Predicción: Positivo (Etiqueta Real: Positivo)

Tweet